In [1]:
import json
import pandas as pd
import redis

# 連接Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# 獲取最新的k線數據
latest_kline = r.get('latest_kline')

# 如果有數據的話
if latest_kline:
    kline_data = json.loads(latest_kline)
    print("最新的K線數據:")
    print(kline_data)

最新的K線數據:
{'symbol': 'SPOT_BTC_USDT', 'type': '5m', 'open': 96182.7, 'close': 96189.26, 'high': 96200.1, 'low': 96182.7, 'volume': 0.509165, 'amount': 48981.0157846, 'startTime': 1733313600000, 'endTime': 1733313900000}


In [ ]:
import asyncio
from redis import asyncio as aioredis
import pandas as pd
from io import StringIO

async def monitor_atr_and_price():
    """簡單的監控函數，只顯示當前 ATR 和價格"""
    try:
        # 連接到本地的 Redis 服務器
        redis_client = await aioredis.from_url("redis://localhost:6379")
        
        while True:
            # 從 Redis 獲取處理後的 K 線數據
            processed_klines = await redis_client.get('processed_klines')
            if processed_klines:
                # 將數據轉換為 DataFrame
                klines_str = processed_klines.decode('utf-8') if isinstance(processed_klines, bytes) else processed_klines
                df = pd.read_json(StringIO(klines_str))
                
                if not df.empty:
                    # 獲取最新的 ATR 和價格
                    current_atr = df['atr'].iloc[-1]
                    current_price = df['close'].iloc[-1]
                    
                    print(f"\nATR: {current_atr:.4f}")
                    print(f"當前價格: {current_price:.2f}")
            
            # 每 300 秒更新一次數據
            await asyncio.sleep(300)
            
    except Exception as e:
        print(f"監控出錯: {str(e)}")
    finally:
        await redis_client.close()

# 在 Jupyter Notebook 中運行這個監控函數
await monitor_atr_and_price()


ATR: 70.3632
當前價格: 96615.00


In [26]:
import json
import pandas as pd
import redis

# 連接Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# 獲取處理後的DataFrame
processed_data = r.get('processed_klines')
if processed_data:
    df = pd.read_json(processed_data)
    display(df)

/var/folders/57/bvn1r6kj60v_d2chpsvv11480000gn/T/ipykernel_33524/3041318129.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(processed_data)


,timestamp,open,high,low,close,volume,atr,direction,tops,bottoms,signal
0,2024-12-07 01:45:00,100681,100800,100380,100416,1400.707,420.000000,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
1,2024-12-08 00:39:00,99582,99618,99556,99571,44.898,640.000000,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
2,2024-12-08 00:40:00,99597,99631,99571,99611,18.247,446.666667,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
3,2024-12-08 00:41:00,99630,99678,99611,99659,64.762,351.750000,down,"[[3, 3, 99678]]","[[68, 68, 99559]]",-1
4,2024-12-08 00:42:00,99660,99676,99599,99669,54.530,296.800000,down,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
...,...,...,...,...,...,...,...,...,...,...,...
77,2024-12-08 01:56:00,99635,99694,99620,99675,61.015,71.174880,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
78,2024-12-08 01:57:00,99675,99684,99588,99620,34.888,72.948103,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
79,2024-12-08 01:58:00,99609,99629,99584,99629,20.770,70.951810,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
80,2024-12-08 01:59:00,99629,99662,99619,99653,19.031,68.955252,up,"[[3, 3, 99678]]","[[68, 68, 99559]]",0
